In [11]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [31]:
df_areas = pd.read_csv("bayut_area_guides.csv")

In [ ]:
df_areas.head(2)

In [33]:
import json
with open('areas.json', 'r') as f:
    areas_list = json.load(f)

### taking only data related to below areas

In [ ]:
len(areas_list)

In [35]:
df_areas_filtered = df_areas[df_areas['img_name'].isin(areas_list)]

### spilt the clumn on basis of comma

In [ ]:
df_areas_filtered['sub_location'] = ''
df_areas_filtered['location'] = ''
def split_location(text):
    if pd.isna(text):  # Handle NaN values
        return pd.Series({'sub_location': '', 'location': ''})
    if ',' in text:
        sub_loc, loc = text.split(',', 1)  # Split on first comma only
        return pd.Series({'sub_location': sub_loc.strip(), 'location': loc.strip()})
    return pd.Series({'sub_location': '', 'location': text.strip()})

df_areas_filtered[['sub_location', 'location']] = df_areas_filtered['img_name'].apply(split_location)

In [ ]:
df_areas_filtered.head(1)

In [70]:
options = uc.ChromeOptions()
options.headless = True  # Run in headless mode
driver = uc.Chrome(options=options)
data=[]

In [133]:
headers_building= ["about"
                   ,"in-a-nutshell","parking","elevators","security,-central-air-conditioning-and-maintenance-services","gym","lifestyle-amenities","concierge-and-guest-services","pet-friendly","apartment-types","studio-apartments"
                   ,"1-bedroom-apartments","2-bedroom-apartments","most-popular-apartments","sales-trends-in","rental-yields","facilities-in","expected-completion-date","payment-plan"
                   ,"getting-around","bus-stations-near","metro-stations","restaurants-near","supermarkets-near","schools-and-nurseries-near","nearby-buildings","major-landmarks","nearby-attractions"
                   ,"things-to-consider","what-type-of-apartments","what-facilities-are-there","faqs"]

In [80]:
def get_markdown(section,headers):
    markdown_content = []
    current = driver.execute_script('return arguments[0].nextElementSibling', section)
    while True:
        try:
            #print(current.get_attribute('class'))
            # Check if the current element has the markdown-elements class
            if 'markdown-elements' in current.get_attribute('class'):
                markdown_content.append(current.text)
            else:
                current_id = current.get_attribute('id')
                if any(current_id and (current_id.startswith(header.strip('*')) for header in headers)):
                    break
                else:
                    markdown_content.append(current.text)
            
            # Move to the next sibling
            current = driver.execute_script('return arguments[0].nextElementSibling', current)
            # Break if there are no more siblings
            if not current:
                break
                
        except Exception as e:
            print(f"Error: {e}")
            break
    
    #print(f"Found {len(markdown_content)} markdown sections")
    #print("\n".join(markdown_content))
    return markdown_content

In [113]:
areas_buildings=[]
url = "https://www.bayut.com/buildings/dubai/"
driver.get(url)
wait = WebDriverWait(driver, 20)
div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[class="grid auto-cols-max grid-flow-col gap-x-3 gap-y-5 md:grid-flow-row md:grid-cols-4 grid grid-cols-auto grid-rows-auto"]')))
areas_buildings_div = div.find_elements(By.CSS_SELECTOR, "a[rel='noreferrer']")
for item in areas_buildings_div:
    obj={}
    obj["location"]= item.text
    obj["url"] = item.get_attribute("href")
    areas_buildings.append(obj)



In [ ]:
len(areas_buildings)

In [ ]:
body = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div#__next')))
len(body.find_elements(By.CSS_SELECTOR, "a[class='h-10 w-50 overflow-hidden rounded bg-primary-dark/5 text-center text-sm font-bold leading-10 text-primary-dark md:w-auto']"))


In [144]:
import json
with open('locations_url.json', 'r') as f:
    locations_url = json.load(f)

### extract top 30 building name from each location 

In [148]:
buildings=[]
for location_name,building_url in locations_url.items():
    for pageNum in range(1,3):
        
        url = f"https://www.bayut.com{building_url}"
        if pageNum != 1:
            url = f"{url}page/{pageNum}/"
        print(url)
        driver.get(url)
        wait = WebDriverWait(driver, 20)
        div = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div#__next')))
        body = div.find_elements(By.CSS_SELECTOR, "div[class='relative gap-4 inline h-fit w-full rounded border border-gray-100']")
        #print(len(body))  
        for item in body:
            obj={}
            modified_url = item.find_element(By.CSS_SELECTOR, "a[rel='noreferrer'").get_attribute("href")
            building_name = item.find_element(By.CSS_SELECTOR, "span[class='text-md p-2 flex cursor-pointer flex-col items-center justify-center text-center min-h-[64px]']").text
            obj['url']= modified_url
            obj['building_name']= building_name
            obj['location_name']= location_name
            obj['location_url']= building_url
            print(modified_url)
            buildings.append(obj)

https://www.bayut.com/buildings/jumeirah-village-circle-jvc/


MaxRetryError: HTTPConnectionPool(host='localhost', port=56319): Max retries exceeded with url: /session/d1f5f6df5a7d774414523db6c9617e10/execute/sync (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7e0fa8b6f7a0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [ ]:
len(buildings)

In [ ]:
buildings

## extract all building info for all buildings

In [ ]:
building_info=[]
for item in buildings:
    print(f"{item["url"]}")
    driver.get(f"{item["url"]}")
    obj={}
    wait = WebDriverWait(driver, 20)
    body = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div#__next')))
    #print('Jumeirah Village Circle (JVC)')
    obj["area_name"] = item["location_name"]
    obj["building_name"] = item["building_name"]
    obj["url"] = item["url"]
    for header in headers_building:
        #print(header)
        #print(f'div[id^="{header}"]')
        try:
            section = body.find_element(By.CSS_SELECTOR, f'div[id^="{header}"]')
            if section:
                obj[header] = "\n".join(get_markdown(section, headers_building))
        except Exception as e:
            print(f"Could not find section for header: {header}")
            obj[header] = ""  # Set empty string for missing sections
    
    building_info.append(obj)


In [ ]:
len(building_info)

In [143]:
df_building_info = pd.DataFrame(building_info)
df_building_info.to_csv("bayut_building_info_JVC_furjan.csv")